In [ ]:
import efinance as ef
import plotly.express as px
import time
import pandas as pd

In [ ]:
#ef.api.stock.config.EASTMONEY_STOCK_BASE_INFO_FIELDS
#ef.api.stock.config.EASTMONEY_STOCK_DAILY_BILL_BOARD_FIELDS
#ef.api.stock.config.HERE
#print(ef.stock.get_members.__doc__)

## market-level command

In [ ]:
# get dates of available financial reports
all_filings_dates = ef.stock.get_all_report_dates()

In [ ]:
#print(ef.stock.get_all_company_performance.__doc__)
# get data of all stocks for a given fiscal quarter
#all_financials_df = ef.stock.get_all_company_performance()
#all_financials_df.head()

### 批量实时行情数据
可提供全市场最新的行情数据，速度较快，可供滚动抓取。

In [ ]:
{
    None: "沪深京A股市场行情",
    "沪深A股": "沪深A股市场行情",
    "沪A": "沪市A股市场行情",
    "深A": "深市A股市场行情",
    "北A": "北证A股市场行情",
    "可转债": "沪深可转债市场行情",
    "期货": "期货市场行情",
    "创业板": "创业板市场行情",
    "美股": "美股市场行情",
    "港股": "港股市场行情",
    "中概股": "中国概念股市场行情",
    "新股": "沪深新股市场行情",
    "科创板": "科创板市场行情",
    "沪股通": "沪股通市场行情",
    "深股通": "深股通市场行情",
    "行业板块": "行业板块市场行情",
    "概念板块": "概念板块市场行情",
    "沪深系列指数": "沪深系列指数市场行情",
    "上证系列指数": "上证系列指数市场行情",
    "深证系列指数": "深证系列指数市场行情",
    "ETF": "ETF 基金市场行情",
    "LOF": "LOF 基金市场行情"
}

# please reformat the dict above
quotes_now = ef.stock.get_realtime_quotes()

## 个股级别数据

In [ ]:
ticker = '601888'  # 中国中免
ticker = '688169'  # 石头科技
ticker = '000550'
ticker = ''
#ticker = quotes_now.query("股票名称 == '科沃斯'")['股票代码'].values[0]
print(ticker)

In [ ]:
stock_base_info = ef.stock.get_base_info(ticker)  # 基本信息

In [ ]:
stock_name = stock_base_info.get("股票名称")

### 资金流向数据

In [ ]:
# 龙虎榜
#ef.stock.get_daily_billboard(start_date='2024-01-10', end_date='2024-01-14')

In [ ]:
# 个股分钟级资金流向
flow_intraday = ef.stock.get_today_bill(ticker)
flow_intraday.head()

In [ ]:
px.line(flow_intraday, x='时间', y=['主力净流入', '超大单净流入', '大单净流入', '中单净流入', '小单净流入'])

In [ ]:
# 资金流向历史数据
bill_hist = ef.stock.get_history_bill(ticker)
bill_hist_cumsum = bill_hist.set_index('日期')[['主力净流入','超大单净流入', '大单净流入', '中单净流入', '小单净流入']].cumsum().applymap(lambda x: x/1e8).reset_index()

### 个股实时行情数据
包含成交量价，以及订单簿数据，每30秒更新一次。

In [ ]:
# get the quote snapshot every 5 seconds for 10 times and concatenate them into a dataframe
#dflist = []
#while True:
#    df = ef.stock.get_quote_snapshot(ticker)
#    dflist.append(df)
#    time.sleep(5)
#    if len(dflist) == 10:
#        break
#df = pd.concat(dflist, axis=1)

quote_snapshot = ef.stock.get_quote_snapshot(ticker)
quote_snapshot

In [ ]:
#ef.stock.get_belong_board(ticker)

### 个股/批量历史行情
1. 可精细到分钟级
2. 提供复权选项

In [ ]:
quote_hist = ef.stock.get_quote_history(
    stock_codes=ticker,
    klt=101,
    fqt=1,
)
quote_hist.head()

In [ ]:
# 日内分钟级行情
last_trading_day = quotes_now.query("股票代码 == @ticker")['最新交易日'].values[0]
quote_intraday = ef.stock.get_quote_history(
    stock_codes=ticker,
    beg=last_trading_day.replace('-', ''),
    klt=1,
    fqt=0,
)

In [ ]:
quote_intraday

### 分析

In [ ]:
flow_renamer = {
    '主力净流入': "prime_order_net_inflow", 
    '超大单净流入': "super_order_net_inflow", 
    '大单净流入': "big_order_net_inflow", 
    '中单净流入': "medium_order_net_inflow", 
    '小单净流入': "small_order_net_inflow",
    "时间": "time"
}
flow_renamer_r = {v: k for k, v in flow_renamer.items()}
flow_en = flow_intraday\
    .rename(columns=flow_renamer)\
    .set_index('time')\
    .loc[:, ["prime_order_net_inflow", "super_order_net_inflow", 
             "big_order_net_inflow", "medium_order_net_inflow", 
             "small_order_net_inflow"]]

In [ ]:
quote_renamer = {
    '日期': 'time', '开盘':'open', '收盘': 'close', '最高': 'high', '最低': 'low', 
    '成交量': 'volume', '成交额':'price*volume', '换手率': 'turnover',
    '涨跌幅': 'change_pct', '涨跌额': 'change'
}
quote_renamer_r = {v: k for k, v in quote_renamer.items()}
quote_en = quote_intraday.rename(columns=quote_renamer).set_index('time').drop(columns=['股票名称','股票代码','振幅'])

In [ ]:
intraday_min = pd.concat([quote_en, flow_en], axis=1)\
    .rename(pd.to_datetime)\
    .assign(min_into_day=lambda df: [dt.hour * 60 + dt.minute for dt in df.index])

In [ ]:
# scatter plot of intraday data
xvar = "prime_order_net_inflow"
yvar = "change_pct"
cvar = 'min_into_day'
hvars = None
fig = px.scatter(intraday_min, x=xvar, y=yvar, color=cvar, hover_data=hvars)
# update layout, set title, labels, and figsize
fig.update_layout(
    title=f"{stock_name} {xvar} vs {yvar}",
    xaxis_title=xvar,
    yaxis_title=yvar,
    width=800,
    height=600,
    template="plotly_dark" 
)
fig.show()

In [ ]:
fig = px.line(flow_intraday.rename(columns=flow_renamer_r), 
              x='时间', y=['主力净流入','超大单净流入', '大单净流入', '中单净流入', '小单净流入'])
fig2 = px.line(quote_intraday, x='日期', y=['收盘'], template='plotly_dark')

from plotly.subplots import make_subplots
figs = make_subplots(rows=1, cols=2)
figs.add_trace(fig2.data[0], row=1, col=1)
for trace in fig.data:
    figs.add_trace(trace=trace, row=1, col=2)
figs.update_layout(height=600, width=1200, title_text="Side By Side Subplots", template='plotly_dark')

figs.update_layout(
    title_text='{0}({1})分钟级资金流向与成交价'.format(stock_name, ticker), 
    xaxis_title="时间", 
    yaxis_title="收盘价", 
    xaxis2_title="时间", 
    yaxis2_title="净流入")

# ignore the middle part 
#figs.update_xaxes(range=[0, 240], row=1, col=1)

figs.show()

In [ ]:
bill_hist_cumsum['日期'].min()

In [ ]:
hist_data = pd.merge(
    quote_hist[['股票名称','日期','收盘','成交额','涨跌幅','换手率']],
    bill_hist[['日期','主力净流入','超大单净流入', '大单净流入', '中单净流入', '小单净流入']],
    on='日期', how='inner'
)

In [ ]:
hist_data.head()

In [ ]:
# turn hist_data wide to long format so that the new dataframe has [日期，资金净流入]
hist_flow_long = hist_data\
    .melt(id_vars=['日期','股票名称','收盘','成交额','涨跌幅','换手率'], 
          value_vars=['主力净流入','超大单净流入', '大单净流入', '中单净流入', '小单净流入'],
          var_name='资金类型',
          value_name='资金净流入')\
    .sort_values('日期')\
    .assign(日期=lambda df: pd.to_datetime(df['日期']))\
    .assign(资金类型=lambda df: df['资金类型'].apply(lambda y : y.replace('净流入','')))\
    .assign(资金净流入=lambda df: df['资金净流入']/1e8)

In [ ]:
from plotly.subplots import make_subplots

fig_price = px.line(hist_data, x='日期', y=['收盘'], 
                    #template='plotly_dark', 
                    color_discrete_sequence=['white'])
fig_flow = px.line(bill_hist_cumsum, x='日期', y=['主力净流入','超大单净流入', '大单净流入', '中单净流入', '小单净流入'])
fig_scatter = px.scatter(hist_flow_long, x='资金净流入', y='涨跌幅', color='资金类型', 
                         hover_data=['日期','成交额','换手率'],
                         #template='plotly_dark', 
                         color_discrete_sequence=['#FFD300','#FF0800','#00A86B','#0080FF','#AF69EF'])

figs = make_subplots(rows=1, cols=3)
figs.add_trace(fig_price.data[0], row=1, col=1)
for trace in fig_flow.data:
    figs.add_trace(trace=trace, row=1, col=2)
for trace in fig_scatter.data:
    figs.add_trace(trace=trace, row=1, col=3)

figs.update_layout(height=600, width=1500, title_text="Side By Side Subplots", template='plotly_dark')
figs.update_layout(
    title_text='{0}({1}) 历史资金流向与价格趋势'.format(stock_name, ticker), 
    xaxis_title="时间", 
    yaxis_title="收盘价", 
    xaxis2_title="日期", 
    yaxis2_title="净流入(亿元)",
    xaxis3_title="资金净流入(亿元)",
    yaxis3_title="涨跌幅（%）")

# place legend at relevant positions
figs.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

figs.show()

## 指数数据

In [ ]:
index_ticker = {
    'CSI300': '000300',  # 沪深300
    'CSI1000': '000852',  # 中证1000
}

In [ ]:
# 获取指数成分股
index_members = ef.stock.get_members(index_ticker['CSI300'])  # 沪深300成分股

In [ ]:
# 通过并行计算，获取沪深300成分股的基本信息
index_const_base_info=ef.stock.get_base_info(index_members['股票代码'].values.tolist())